In [ ]:
import os
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

In [ ]:
audio_file = "/content/drive/My Drive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/TrimmedMusicCaps/_0-2meOf9qY_trimmed.wav"
waveform, sr = torchaudio.load(audio_file)
if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
if sr != 16000:  # Resample to 16kHz
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    waveform = resampler(waveform)

# Process audio
inputs = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True)
inputs = inputs.to(device)

# Extract embeddings
with torch.no_grad():
    outputs = model(**inputs)
    frame_embeddings = outputs.last_hidden_state  # Shape: (batch_size, frames, embedding_dim)
    pooled_features = frame_embeddings.mean(dim=0)

print(frame_embeddings.shape)  # Example: (1, 300, 768)
print(pooled_features.shape)  # Example: (768,)

torch.Size([1, 349, 768])
torch.Size([349, 768])


In [ ]:
audio_folder = "/content/drive/My Drive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/TrimmedMusicCaps"
output_file = "/content/drive/My Drive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/Wav2Vec/MusicCaps_time_embeddings.npz"
os.makedirs(audio_folder, exist_ok=True)

# Dictionary to store embeddings
embeddings_dict = {}

# Function to process a single audio file
def process_audio_file(audio_path):
    try:
        # Load audio
        waveform, sr = torchaudio.load(audio_path)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        if sr != 16000:  # Resample to 16kHz
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            waveform = resampler(waveform)

        # Process audio to get embeddings
        inputs = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True)
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            frame_embeddings = outputs.last_hidden_state  # Shape: (batch_size, frames, embedding_dim)

        return frame_embeddings.squeeze(0).cpu().numpy()  # Remove batch dimension
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

idx = 0
# Process all audio files in the folder
for audio_file in os.listdir(audio_folder):

    if audio_file.endswith(".wav"):  # Adjust extension if needed
        audio_path = os.path.join(audio_folder, audio_file)
        # print(f"Processing {audio_file}...")
        embeddings = process_audio_file(audio_path)
        audio_file_name = (audio_file.split("/")[-1]).split(".")[0]
        if embeddings is not None:
            embeddings_dict[audio_file_name] = embeddings
            # print(f"Processed {audio_file}")
            idx += 1
            if idx % 100 == 0:
                print(f"Processed {idx} files...")
        else:
            print(f"Failed to process {audio_file}")

# Save all embeddings to a single .npz file
if embeddings_dict:
    np.savez_compressed(output_file, **embeddings_dict)
    print(f"All embeddings saved to {output_file}")
else:
    print("No embeddings were processed.")

Processed 100 files...
Processed 200 files...
Processed 300 files...
Processed 400 files...
Processed 500 files...
Processed 600 files...
Processed 700 files...
Processed 800 files...
Processed 900 files...
Processed 1000 files...
Processed 1100 files...
Processed 1200 files...
Processed 1300 files...
Processed 1400 files...
Processed 1500 files...
Processed 1600 files...
Processed 1700 files...
Processed 1800 files...
Processed 1900 files...
Processed 2000 files...
Processed 2100 files...
Processed 2200 files...
Processed 2300 files...
Processed 2400 files...
Processed 2500 files...
Processed 2600 files...
Processed 2700 files...
Processed 2800 files...
Processed 2900 files...
Processed 3000 files...
Processed 3100 files...
Processed 3200 files...
Processed 3300 files...
Processed 3400 files...
Processed 3500 files...
Processed 3600 files...
Processed 3700 files...
Processed 3800 files...
Processed 3900 files...
Processed 4000 files...
Processed 4100 files...
Processed 4200 files...
P

In [ ]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

AIME_wav2vec_path = "/content/drive/MyDrive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/Wav2Vec/old_MusicCaps_time_embeddings.npz"
data = np.load(AIME_wav2vec_path)

keys = list(data.keys())  # Extract the keys (file names)
embeddings = [data[key] for key in keys]  # Extract corresponding embeddings
keys_array = np.array(keys, dtype=str)  # Ensure keys are strings
embeddings_array = np.array(embeddings)  # Convert embeddings to a single array

# Save to a new .npz file
output_file = "/content/drive/MyDrive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/Wav2Vec/MusicCaps_time_embeddings.npz"
np.savez(output_file, file_names=keys_array, time_embeddings=embeddings_array)

print(f"Converted embeddings saved to {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Converted embeddings saved to /content/drive/MyDrive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/Wav2Vec/MusicCaps_time_embeddings.npz


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

output_file = "/content/drive/MyDrive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/Wav2Vec/MusicCaps_time_embeddings.npz"
loaded = np.load(output_file)
print(loaded.keys())

Mounted at /content/drive
KeysView(NpzFile '/content/drive/MyDrive/MIT-Education/Junior Year/Music ML/Ananya-Kimaya Final Project/Sample Audios/Wav2Vec/MusicCaps_time_embeddings.npz' with keys: file_names, time_embeddings)


In [ ]:
len(loaded['file_names'])

5356

In [ ]:
len(loaded['time_embeddings'])

5356

In [ ]:
len(loaded['time_embeddings'][0])

349

In [ ]:
len(loaded['time_embeddings'][0][0])

768